## Q4)  a-e

In [1]:
import autograd.numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from autograd import grad
from autograd import elementwise_grad as egrad
import torch
#from torch.autograd import grad
#from torch.autograd import Variable
from sklearn.preprocessing import StandardScaler


## Obtainig Data

In [2]:
def RMS(y1,y2):
    #calculating RMS
    y1=np.transpose(y1)[0]
    y2=np.transpose(y2)[0]
    rms=(sum(((y1-y2)**2))/len(y1))**0.5
    return rms

filname='Real estate valuation data set.xlsx'
df=pd.read_excel(filname)
data=np.array(df)
train_index=np.random.choice(np.array(range(414)),290, replace=False)        #test:train=30:70=124:290
train_set=np.array([data[i] for i in train_index])                          #random shuffling
test_set=[]
for i in range(414):
    if i not in train_index:
        test_set.append(data[i])
test_set=np.array(test_set)


train_X=train_set[:,1:7]
train_Y=np.transpose(np.array([train_set[:,-1]]))
test_X=test_set[:,1:7]
test_Y=np.transpose([test_set[:,-1]])                                #ground_truth

#normalize the data to prevent overflow in gradient descent (reference: )
scaler = StandardScaler()
train_X = scaler.fit_transform(train_X)
test_X = scaler.transform(test_X)


##  Defining Required Functions for Linear Regression

In [3]:
def normalEquationRegression(X, y):
    X=np.insert(X,0,values=np.ones(len(X)),axis=1)
    X_=np.transpose(X)
    A=np.matmul(X_,X)
    theta=0
    if np.linalg.det(A)==0:
        theta=normalEquationRegression(X[:,1:], y)
    else:
        A=np.linalg.inv(A)
        B=np.matmul(X_,y)
        theta=np.matmul(A,B)
    return theta

def gradientDescentRegression(X, y, alpha = 0.00001, iteration=100000):
    X=np.insert(X,0,values=np.ones(len(X)),axis=1)
    theta=np.random.randn(len(X[0]),1)
    X_t=np.transpose(X)
    for i in range(iteration):
        y_=np.matmul(X,theta)
        error=y_-y
        theta=theta-(alpha/len(y))*np.matmul(X_t,error)     
    return theta


def Error(theta):     #requires global declaration of y
    y=train_Y
    Y_=np.matmul(np.insert(train_X,0,values=np.ones(len(train_X)),axis=1),theta)
    return (1/len(Y_))*(Y_-y)**2
def gradientDescentAutogradRegression(X, y, alpha = 0.00001,iteration=100000):
    X=np.insert(X,0,values=np.ones(len(X)),axis=1)
    theta=np.random.randn(len(X[0]),1)
    derivative=egrad(Error)
    for i in range(iteration):
        #y_=np.matmul(X,theta)
        theta=theta-alpha*np.array(derivative(theta))
    return theta

def gradientDescentPyTorchRegression(X, y, alpha = 0.1):
    N=torch.tensor(len(y))
    X=np.insert(X,0,values=np.ones(len(X)),axis=1)
    X=torch.tensor(X, requires_grad=True)
    y=torch.tensor(y, requires_grad=True)
    alpha=torch.tensor(alpha)
    theta=torch.randn((len(X[0]),1),requires_grad=True)
    iteration=1000
    
    for i in range(iteration):
        y_=torch.mm(torch.tensor(X),theta)
        error=(1/N)*torch.mm((torch.mm(torch.tensor(X),theta)-y),(torch.mm(torch.tensor(X),theta)-y).t())
        error.backward()
        theta=theta-alpha*theta.grad
    return theta


## Obtaining results

In [4]:
#theta_normal=normalEquationRegression(train_X,train_Y)
#theta_grad=gradientDescentRegression(train_X, train_Y)
#theta_autograd=gradientDescentAutogradRegression(train_X, train_Y)
torch.set_default_tensor_type('torch.DoubleTensor')
theta_torch=gradientDescentPyTorchRegression(train_X, train_Y)

/home/shivji/.local/lib/python3.5/site-packages/ipykernel_launcher.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/shivji/.local/lib/python3.5/site-packages/ipykernel_launcher.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


RuntimeError: grad can be implicitly created only for scalar outputs

In [ ]:
theta_normal

In [ ]:
theta_autograd

In [ ]:
theta_grad

In [ ]:
#theta_pytorch

In [ ]:
X=np.insert(test_X,0,values=np.ones(len(test_X)),axis=1)
print("The rmse for linear regression model as calculated in Q3 is 9.05")
print("")
print("The rmse for Normal Eqn, Grad.Descent, AutoGrad are: ",)
print("")
for theta in [theta_normal, theta_grad, theta_autograd]:
    Y=np.matmul(X,theta)
    rmse=RMS(Y,test_Y)
    print(rmse,'',)

In [ ]:
torch.tensor([[5,5,6]]).t()